# Importation des bibliothèques

In [1]:
%%capture --no-display

import warnings

warnings.filterwarnings("ignore")

import calendar
import datetime as dt
import re

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)

from io import BytesIO

try:
    import openpyxl as pyxl
    from efc.interfaces.iopenpyxl import OpenpyxlInterface
    from fuzzywuzzy import fuzz, process
except ImportError or ModuleNotFoundError:
    !pip install openpyxl
    !pip install python-Levenshtein
    !pip install fuzzywuzzy
    !pip install excel-formulas-calculator
    import openpyxl as pyxl
    from efc.interfaces.iopenpyxl import OpenpyxlInterface
    from fuzzywuzzy import fuzz, process

In [2]:
%%capture
import sys

path = "/home/jovyan/workspace/Fichier Suivi de Stock/code/pipelines/"
if path not in sys.path:
    sys.path.append(path)

import inspect
from importlib import reload

import generate_stock_tracking_file as gstf
from database_operations import db_ops

reload(gstf)
reload(db_ops)

<h1> Données d'entrée</h1>
Ce fichier est généré individuellement pour chaque programme de santé

* `Rapport Feedback`: principalement les feuilles `Etat de Stock` et `StockParRegion`
* `Le plan d'approvisionnement`: du programme pour lequel on concoit le fichier
* `Etats de stock mensuels`: fichier individuel pour le programme pour lequel on concoit le fichier

# Initialisation des variables `programme`, `date_report`

In [3]:
programme = 'PNLP'
date_report = '01/12/2024'
date_report_format = pd.to_datetime(date_report, format= '%d/%m/%Y').strftime('%Y-%m-%d')

# Importation des classeurs

In [4]:
%%time
# Template utilisé pour le formating des fichiers
wb_template = pyxl.load_workbook(
    filename="/home/jovyan/workspace/Fichier Suivi de Stock/code/pipelines/generate_stock_tracking_file/Template Fichier Suivi de Stock/Fichier Suivi de Stock Template.xlsx"
)

# Etats de stock mensuels
wb_etat_stock = pyxl.load_workbook(
    filename="/home/jovyan/workspace/Fichier Suivi de Stock/code/pipelines/generate_stock_tracking_file/Template Fichier Suivi de Stock/Etat du stock et de distribution PNLP fin Janvier 2025.xlsx"
)

CPU times: user 2.75 s, sys: 33.3 ms, total: 2.79 s
Wall time: 2.86 s


# Génération du classeur

## Mise à jour des données en se basant sur la source `Etat de Stock Mensuel`

In [5]:
%%time
wb_template = gstf.update_sheets_etat_mensuel(wb_etat_stock, wb_template, programme, date_report)

CPU times: user 24.3 s, sys: 47.2 ms, total: 24.3 s
Wall time: 24.4 s


## Mise à jour des données en se basant sur la source `Rapport Feedback`

In [6]:
%%script false --no-raise-error
%%time
file_path = "/home/jovyan/workspace/automating_dap_tools/data/feedback_report/generate_feedback_report/RapportFeedBack-JANVIER-2025.xlsx"
with (open(file_path, "rb") as file_handle, BytesIO(file_handle.read()) as buffer):
    wb_fbr = pyxl.load_workbook(filename=buffer)
    buffer.seek(0)
    df_etat_stock = pd.read_excel(buffer, sheet_name="ETAT DU STOCK", engine="openpyxl")

# Wall time: 1min 36s

In [7]:
%%time
## Rapport Feedback
wb_fbr = pyxl.load_workbook(
    "/home/jovyan/workspace/automating_dap_tools/data/feedback_report/generate_feedback_report/RapportFeedBack-JANVIER-2025.xlsx"
)

df_etat_stock = pd.read_excel(
    "/home/jovyan/workspace/automating_dap_tools/data/feedback_report/generate_feedback_report/RapportFeedBack-JANVIER-2025.xlsx",
    sheet_name="ETAT DU STOCK",
)

df_etat_stock.head(1)

CPU times: user 1min 40s, sys: 2 s, total: 1min 42s
Wall time: 1min 43s


,Code_Pro,CODE,PROGRAMME,SOUS-PROGRAMME,PERIODE,REGION,DISTRICT,CODE ETS,STRUCTURE,TYPE DE STRUCTURE,CATEGORIE PRODUIT,PRODUIT,UNITE DE RAPPORTAGE,STOCK INITIAL,QUANTITE RECUE,QUANTITE UTILISEE,PERTES ET AJUSTEMENT,JOURS DE RUPTURE,SDU,CMM ESIGL,CMM gestionnaire,QUANTITE PROPOSEE,QUANTITE COMMANDEE,QUANTITE APPROUVEE,MSD,ETAT DU STOCK,BESOIN COMMANDE URGENTE,BESOIN TRANSFERT IN,QUANTITE A TRANSFERER OUT,CATEGORIE_DU_PRODUIT,Commandes du mois précédent,Etat du stock du mois précédent
0,3050340_PNLP,3050340,PNLP,PNLP-MEDICAMENTS ET INTRANTS,JANVIER 2025,ABIDJAN 2,COCODY-BINGERVILLE,60300100,CSU COM ANONO VILLAGE,ESCOM,Produit traceur,SULFADOXINE+PYRIMETHAMINE 500/25 mg comp. UN -,COMPRIME,1140,0,1140,0,0,0,1157,1125.0,4628,4500,4500,0.0,RUPTURE,4500.0,1125.0,NaN,PRODUITS PNLP,NaN,NaN


In [8]:
%%time

wb_template = gstf.update_sheet_etat_stock(wb_template, df_etat_stock.loc[df_etat_stock.PROGRAMME==programme].copy())

wb_template = gstf.update_sheet_stock_region(wb_template, wb_fbr, programme)

CPU times: user 11.2 s, sys: 5.92 ms, total: 11.2 s
Wall time: 11.2 s


## Mise à jour des informations de la feuille `Annexe 1 - Consolidation`

In [9]:
db_ops.reload_connection()

schema_name = "suivi_stock"

In [10]:
# type(db_ops.civ_engine)
# df_produit = pd.read_sql(
#     f"""
#     SELECT prod.*, st.stock_theorique_mois_precedent
#     FROM {schema_name}.stock_track st
#     INNER JOIN {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
#     """,
#     db_ops.civ_engine,
# ).drop_duplicates()

# df_produit.head(2)

In [11]:
%%script false --no-raise-error

df_dmm_global = pd.read_sql(
    f"""
    SELECT prod.*, st_dmm.* FROM {schema_name}.stock_track_dmm st_dmm
    INNER JOIN {schema_name}.dim_produit_stock_track prod ON st_dmm.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    WHERE prod.programme='{programme}'""",
    db_ops.civ_engine,
)

df_dmm_global["date_report"] = pd.to_datetime(df_dmm_global["date_report"])

df_dmm_histo = pd.read_sql(
    f"""
    SELECT prod.*, st_dmm_histo.date_report, st_dmm_histo.date_report_prev, st_dmm_histo.dmm  FROM {schema_name}.stock_track_dmm_histo st_dmm_histo
    INNER JOIN {schema_name}.dim_produit_stock_track prod ON st_dmm_histo.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    WHERE prod.programme='{programme}'""",
    db_ops.civ_engine,
)
df_dmm_histo["date_report"] = pd.to_datetime(df_dmm_histo["date_report"])

df_cmm_global = pd.read_sql(
    f"""
    SELECT prod.*, st_cmm.* FROM {schema_name}.stock_track_cmm st_cmm
    INNER JOIN {schema_name}.dim_produit_stock_track prod ON st_cmm.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    WHERE prod.programme='{programme}'""",
    db_ops.civ_engine,
)
df_cmm_global["date_report"] = pd.to_datetime(df_cmm_global["date_report"])

df_cmm_histo = pd.read_sql(
    f"""
    SELECT prod.*, st_cmm_histo.date_report, st_cmm_histo.date_report_prev, st_cmm_histo.cmm  FROM {schema_name}.stock_track_cmm_histo st_cmm_histo
    INNER JOIN {schema_name}.dim_produit_stock_track prod ON st_cmm_histo.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    WHERE prod.programme='{programme}'""",
    db_ops.civ_engine,
)
df_cmm_histo["date_report"] = pd.to_datetime(df_cmm_histo["date_report"])

In [12]:
wb_template = gstf.update_sheet_annexe_1(
    wb_template, programme, schema_name, db_ops.civ_engine, date_report
)

In [13]:
# %%time
# ws_annexe_1, df_prod = gstf.update_first_informations(
#     wb_temp=wb_template,
#     df_produit=df_produit,
#     date_report= date_report,
# )

## Mise à jour des données du `Plan d'approvisionnement`

In [14]:
file_path_plan_approv, file_map_prod = (
    "/home/jovyan/workspace/automating_dap_tools/data/stock_tracking_file/PNLP/Plan d'Appro/",
    "/home/jovyan/workspace/automating_dap_tools/data/stock_tracking_file/mapping_produits_qat_sage/Mapping QAT_SAGEX3.xlsx"
)

In [15]:
def process_pa_files(file_path_plan_approv, file_map_prod, programme):
    import os
    # Liste pour accumuler les données
    data = []

    # Fonction pour traiter un fichier unique
    def _process_pa_file(fichier):
        nonlocal data
        with open(file=fichier) as file:
            for line in file.readlines():
                if len(line.split(',')) == 17:
                    data.append([col.replace('"', '') for col in line.strip().split(',')])

    if os.path.isdir(file_path_plan_approv):
        for root, _, files in os.walk(file_path_plan_approv):
            for file in files:
                if file.endswith('.csv'):
                    _process_pa_file(os.path.join(root, file))
    else:
        _process_pa_file(file_path_plan_approv)

    df_plan_approv = pd.DataFrame(data[1:], columns=data[0])
    
    # Bad index
    bad_index = df_plan_approv.loc[df_plan_approv["ID de produit QAT / Identifiant de produit (prévision)"]=="ID de produit QAT / Identifiant de produit (prévision)"].index

    df_plan_approv.drop(index=bad_index, inplace=True)
        
    for col in ['ID de produit QAT / Identifiant de produit (prévision)', 'ID de l`envoi QAT']:
        try:
            df_plan_approv[col] = df_plan_approv[col].astype('Int64')
        except:
            pass
    
    for col in ['Coût unitaire de produit (USD)', 'Coût du fret (USD)', 'Quantité', 'Coût total (USD)']:
        try:
            df_plan_approv[col] = df_plan_approv[col].astype(float)
        except:
            pass
    
    try:
        df_plan_approv['date de réception'] = df_plan_approv['date de réception'].apply(
            lambda date_str: datetime.strptime(date_str, '%d-%b-%Y'))
    except:
        pass

    # Nettoyage des espaces blancs dans les données
    df_plan_approv = df_plan_approv.apply(lambda x: x.str.strip() if x.dtype == "object" else x)    

    # Charger le fichier de mappage des produits
    df_map_prod = pd.read_excel(file_map_prod, sheet_name=programme)  # ou pd.read_csv selon le type
    df_map_prod.columns = df_map_prod.columns.str.replace('Ã©', 'é').str.replace('â', '').str.rstrip().str.lstrip()
    
    df_map_prod.rename(columns={
        "Code QAT": "ID de produit QAT / Identifiant de produit (prévision)",
        "Code standard national": "Standard product code"
    }, inplace=True)

    
    df_map_prod = df_map_prod.drop_duplicates()

    df_map_prod = df_map_prod.loc[
        (df_map_prod['Standard product code'].notna()) & (df_map_prod['Standard product code'] != 'ND')
    ]
    df_map_prod.sort_values(["ID de produit QAT / Identifiant de produit (prévision)", "Acronym"], inplace =True)
    
    df_map_prod = df_map_prod.drop_duplicates(subset="ID de produit QAT / Identifiant de produit (prévision)", keep='last')

    assert df_plan_approv.merge(
        df_map_prod[["ID de produit QAT / Identifiant de produit (prévision)", "Standard product code", "Acronym"]],
        on='ID de produit QAT / Identifiant de produit (prévision)',
        how='left').shape[0] == df_plan_approv.shape[0]

    df_plan_approv = df_plan_approv.merge(
        df_map_prod[["ID de produit QAT / Identifiant de produit (prévision)", "Standard product code", "Acronym"]],
        on='ID de produit QAT / Identifiant de produit (prévision)',
        how='left')

    return df_plan_approv

In [16]:
df_plan_approv = process_pa_files(file_path_plan_approv, file_map_prod, programme)

In [17]:
df_pa = df_plan_approv.rename(
    columns={
        "Standard product code": "Standard product code",
        "ID de l`envoi QAT": "ID de l`envoi QAT",
        "ID de produit QAT / Identifiant de produit (prévision)": "ID de produit QAT",
        "Produit (planification) / Produit (prévision)": "Produits",
        "Agent d`approvisionnement": "Centrale d'achat",
        "Source de financement": "Source Financement",
        "État": "Status",
        "Quantité": "Quantite",
        "date de réception": "DATE",
        "Coût unitaire de produit (USD)": "Cout des Produits",
        "Coût du fret (USD)": "Couts du fret",
        "Coût total (USD)": "Couts totaux",
    }
)

df_pa["Facteur de conversion de QAT vers SAGE"] = 1

df_pa["DATE"] = pd.to_datetime(df_pa["DATE"], format='%d-%b-%Y', errors='coerce')

In [18]:
# import inspect
# print(inspect.getsource(gstf.update_sheet_plan_approv))
wb_template = gstf.update_sheet_plan_approv(wb_template, df_pa)

## Mise à jour des données `Annexe 2 - Suivi de Stock`

In [19]:
wb_template = gstf.update_sheet_annexe_2(wb_template, df_pa, date_report)

## Mise à jour des données en se basant sur la source `Prévision`

In [20]:
# wb_template = pyxl.load_workbook(
#    filename=f"/home/jovyan/workspace/Fichier Suivi de Stock/code/pipelines/generate_stock_tracking_file/Export Fichier Suivi de Stock/Fichier Suivi de Stock {programme}.xlsx"
# )

In [22]:
df_produit = pd.read_sql(
    f"""
    SELECT prod.*, st.stock_theorique_mois_precedent
    FROM {schema_name}.stock_track st
    INNER JOIN {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    """,
    db_ops.civ_engine,
).drop_duplicates()

In [23]:
df_prod = df_produit[["code_produit", "type_produit", "designation", "designation_acronym"]]

df_prod["designation"] = df_prod.apply(
    lambda row: row["designation_acronym"] if not pd.isna(row["designation_acronym"]) else row["designation"], axis=1
)
df_prod = df_prod.drop(columns="designation_acronym")

In [24]:
wb_template = gstf.update_sheet_prevision(wb_template, date_report, df_prod)

In [25]:
%%time
wb_template.save(
    filename=f"/home/jovyan/workspace/Fichier Suivi de Stock/code/pipelines/generate_stock_tracking_file/Export Fichier Suivi de Stock/Fichier Suivi de Stock {programme}.xlsx"
)

CPU times: user 4.78 s, sys: 144 ms, total: 4.92 s
Wall time: 5.75 s


In [ ]:
gstf.

In [24]:
pyxl.utils.column_index_from_string("BI"), pyxl.utils.column_index_from_string("ck")

(61, 89)

In [7]:
pyxl.utils.get_column_letter(27)

'AA'

In [10]:
pyxl.utils.quote_sheetname("Plan d'appro")

"'Plan d''appro'"

In [67]:
ws_annexe_1["CW3"].value, ws_annexe_1["CX3"].value, ws_annexe_1["CY3"].value

('Nbre de mois de considérés',
 'Consommations enregistrées sur les mois de considérés',
 'CMM Calculée en fin du mois')

In [53]:
df_dmm_histo = pd.read_sql(
    f"""
    SELECT prod.*, st_dmm_histo.date_report, st_dmm_histo.date_report_prev, st_dmm_histo.dmm  FROM {schema_name}.stock_track_dmm_histo st_dmm_histo
    INNER JOIN {schema_name}.dim_produit_stock_track prod ON st_dmm_histo.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    WHERE prod.programme='{programme}'""",
    db_ops.civ_engine,
)

In [50]:
df_dmm_histo["date_report_prev"] = pd.to_datetime(df_dmm_histo["date_report_prev"])

In [64]:
pd.to_datetime("2024-07-01", format="%Y-%m-%d").strftime("%Y-%m-%d") in df_dmm_histo.loc[df_dmm_histo.id_dim_produit_stock_track_pk==1, "date_report_prev"].values

False

In [59]:
df_dmm_histo.loc[df_dmm_histo.id_dim_produit_stock_track_pk==1, "date_report_prev"].values

array([datetime.date(2024, 7, 1), datetime.date(2024, 8, 1),
       datetime.date(2024, 9, 1), datetime.date(2024, 10, 1),
       datetime.date(2024, 11, 1), datetime.date(2024, 12, 1)],
      dtype=object)

# Extraction des informations sur la feuille `Rapport`

In [48]:
file_path = "/home/jovyan/workspace/automating_dap_tools/data/stock_tracking_file/folder_by_month/Juin 2024/Fichier Suivi de Stock PNLP - Juin 2024.xlsx"
wb_etat_stock = pyxl.load_workbook(file_path)
ws_rapport = wb_etat_stock['Rapport']

In [49]:
for row in ws_rapport.iter_rows(min_col=2, max_col=gstf.column_index_from_string('P')):
    for cell in row:
        try:
            if cell.value == 'Tableau de revue des diligences':
                infos_revue_diligences = [cell.row, cell.coordinate]
            if cell.value == 'Tableau de consolidation des stocks':
                infos_cons_stocks = [cell.row, cell.coordinate]
            if cell.value == 'Gestion des risques et diligences':
                infos_gest_diligences = [cell.row, cell.coordinate]
            if cell.value == 'Tableau de recommandations et diligences':
                infos_recommandation_diligences = [cell.row, cell.coordinate]
        except:
            continue

In [50]:
infos_revue_diligences, infos_cons_stocks, infos_gest_diligences, infos_recommandation_diligences

([32, 'C32'], [59, 'C59'], [68, 'C68'], [78, 'C78'])

## Tableau de revues des dilligences

In [52]:
df_rev_dilligences = pd.read_excel(file_path,
                                   sheet_name='Rapport', skiprows=infos_revue_diligences[0]+1, usecols='C:O', nrows=20)

df_rev_dilligences = df_rev_dilligences[[col for col in df_rev_dilligences.columns if not col.startswith('Unnamed:')]]

df_rev_dilligences = df_rev_dilligences.loc[df_rev_dilligences.Diligences.notna()].drop(columns='N°')

df_rev_dilligences

,Diligences,Responsables,Délais,Etat d'exécution,Obsevation
0,Distribuer préférentiellement les AP adulte a...,Nouvelle PSP-CI,En continu,NaN,NaN
1,Demander la réalisation d’un inventaire de de...,Nouvelle PSP-CI,2024-06-30 00:00:00,NaN,En attente du rapport d’inventaire
2,Transmettre à la DAP et aux PNS le rapport d’...,Nouvelle PSP-CI,2024-06-30 00:00:00,NaN,Rapport rédigé en attente de signature par le DG


## Tableau de consolidation des stocks

In [57]:
df_cons_stocks = pd.read_excel(file_path, sheet_name='Rapport', 
                               skiprows=infos_cons_stocks[0]+1, usecols='C:O', nrows=infos_gest_diligences[0]-infos_cons_stocks[0]-5)

df_cons_stocks = df_cons_stocks[[col for col in df_cons_stocks.columns if not col.startswith('Unnamed:')]]

df_cons_stocks

,Produits concernés,Ecarts Observés,Justifications des écarts,Actions à mener


## Gestion des risques des diligences

In [58]:
df_gestion_diligences = pd.read_excel(file_path, sheet_name='Rapport',
                                      skiprows=infos_gest_diligences[0]+1, usecols='C:O', nrows=infos_recommandation_diligences[0]-infos_gest_diligences[0]-3)

import functools

list_columns = list(df_gestion_diligences.columns)

for col in list_columns[3:]:
    val = df_gestion_diligences.loc[0, col]
    if not pd.isna(val):
        df_gestion_diligences.rename(
            columns={col: functools.reduce(
                lambda a, b: a if ('Unnamed' not in a) and ('Unnamed' in b) else b,
                list_columns[: list_columns.index(col)+1]) + '_' + val
            }, inplace=True)
        

df_gestion_diligences = df_gestion_diligences.drop(index=[0]).reset_index(drop=True)[[col for col in df_gestion_diligences.columns if not col.startswith('Unnamed:')]]

df_gestion_diligences

,Désignation,Niveau de stock_central,Niveau de stock_périphérique,Niveau de stock_national,risque de rupture ou de péremption,Commandes en Cours_quantité,Commandes en Cours_Date Probable de livraison,Commandes en Cours_Statut commande,Diligences au niveau Central,Diligences au niveau périphérique,Responsable


## Tableau de recommandations des diligences

In [59]:
df_recom_diligences = pd.read_excel(file_path, sheet_name='Rapport', 
                                    skiprows=infos_recommandation_diligences[0]+1, usecols='C:O', nrows=20)

df_recom_diligences = df_recom_diligences[[col for col in df_recom_diligences.columns if not col.startswith('Unnamed:')]]

df_recom_diligences = df_recom_diligences.loc[df_recom_diligences.Diligences.notna()].drop(columns='N°')

df_recom_diligences

,Diligences,Responsables,Délais,Etat d'exécution,Obsevation


# Mise à jour des données en se basant sur la source `Etat de Stock Mensuel`

In [7]:
%%time
wb_template = gstf.update_data_based_on_file_etat_mensuel(wb_etat_stock, wb_template, programme, date_report)

CPU times: user 1min 4s, sys: 115 ms, total: 1min 4s
Wall time: 1min 4s


# Mise à jour des données en se basant sur la source `Rapport Feedback`

Pour le faire on va se baser sur les inputs au préalable du fichier RapportFeebback généré au mois courant

In [8]:
df_etat_stock = pd.read_excel(f'/home/jovyan/workspace/automating_dap_tools/data/feedback_report/generate_feedback_report/RapportFeedBack-JUIN-2024.xlsx',
                              sheet_name='ETAT DU STOCK', skiprows=1)
df_etat_stock.head(2)

,Code_Pro,CODE,PROGRAMME,SOUS-PROGRAMME,PERIODE,REGION,DISTRICT,CODE ETS,STRUCTURE,CATEGORIE PRODUIT,PRODUIT,UNITE DE RAPPORTAGE,STOCK INITIAL,QUANTITE RECUE,QUANTITE UTILISEE,PERTES ET AJUSTEMENT,JOURS DE RUPTURE,SDU,CMM ESIGL,CMM gestionnaire,QUANTITE PROPOSEE,QUANTITE COMMANDEE,QUANTITE APPROUVEE,MSD,ETAT DU STOCK,BESOIN COMMANDE URGENTE,BESOIN TRANSFERT IN,QUANTITE A TRANSFERER OUT,Commandes du mois précédent,Etat du stock du mois précédent,CATEGORIE_DU_PRODUIT
0,3160005_PNN,3160005,PNN,PNN-MEDICAMENTS ET INTRANTS,JUIN 2024,KABADOUGOU,ODIENNE,51900050,DISTRICT SANITAIRE ODIENNE,Produit non traceur,ACIDE FOLIQUE 5 mg comp. BTE/10 BTE -,COMPRIME,0,0,0,0,30,0,0,500.0,0,2000,2000,0.0,RUPTURE,2000.0,500.0,NaN,NaN,NaN,MEDICAMENTS ET PETITS MATERIELS MEDICAL
1,3160005_PNN,3160005,PNN,PNN-MEDICAMENTS ET INTRANTS,JUIN 2024,GBEKE,SAKASSOU,51100060,DISTRICT SANITAIRE SAKASSOU,Produit non traceur,ACIDE FOLIQUE 5 mg comp. BTE/10 BTE -,COMPRIME,0,0,0,0,30,0,0,0.0,0,0,0,NaN,ND,NaN,NaN,NaN,NaN,NaN,MEDICAMENTS ET PETITS MATERIELS MEDICAL


In [9]:
%%time

wb_template = gstf.update_sheet_etat_stock_with_dataframe(wb_template, df_etat_stock.loc[df_etat_stock.PROGRAMME==programme].copy())

wb_template = gstf.update_sheet_stock_region(wb_template, wb_fbr, programme)

CPU times: user 47.1 s, sys: 66.8 ms, total: 47.2 s
Wall time: 47.3 s


# Mise à jour des données du `Plan d'approvisionnement`

Avant d'effectuer la mise à jour de la feuille Annexe 2, il nous faut les informations du plan d'approvisionnement

## Prétraitrement du fichier de plan d'approvisionnement

In [10]:
from datetime import datetime

file_path = "/home/jovyan/workspace/automating_dap_tools/data/stock_tracking_file/folder_by_month/Juin 2024/Plan d'Approvisionnement/PNLS-ARV.csv"
data = []
with open(file=file_path) as file:
    for line in file.readlines():
        if len(line.split(','))==17:
            data.append([col.replace('"', '') for col in line.strip().split(',')])

df_plan_approv = pd.DataFrame(data[1:], columns= data[0])

for col in ['ID de produit QAT / Identifiant de produit (prévision)', 'ID de l`envoi QAT', 'Quantité',]:
    df_plan_approv[col] = df_plan_approv[col].astype(int)
    
for col in ['Coût unitaire de produit (USD)', 'Coût du fret (USD)', 'Coût total (USD)']:
    df_plan_approv[col] = df_plan_approv[col].astype(float)
    
df_plan_approv['date de réception'] = df_plan_approv['date de réception'].apply(lambda date_str: datetime.strptime(date_str, '%d-%b-%Y'))

df_plan_approv = df_plan_approv.map(lambda x: x.lstrip().rstrip() if isinstance(x, str) else x)

del data

df_plan_approv.head()

,ID de produit QAT / Identifiant de produit (prévision),Produit (planification) / Produit (prévision),ID de l`envoi QAT,Commande d`urgence,Commande PGI,Approvisionnement local,N° de commande de l`agent d`approvisionnement,Agent d`approvisionnement,Source de financement,Budget,État,Quantité,date de réception,Coût unitaire de produit (USD),Coût du fret (USD),Coût total (USD),Notes
0,1082,Abacavir/Lamivudine 600/300 mg Tablet 30 Tablets,175651,false,false,false,,Govt,Govt,BGE_2024,Reçu,30000,2024-01-25,270000.00,29700.00,299700.00,
1,2786,Efavirenz 200 mg Dispersible Tablet 90 Tablets,175693,false,false,false,,Govt,Govt,BGE_2023,Reçu,4382,2024-02-13,60164.86,6618.13,66782.99,
2,3157,Fluconazole 200 mg Capsule 10 x 10 Blister Pa...,175698,false,false,false,,GHSC-PSM,PEPFAR,COP23_ARV,Reçu,2674,2024-02-23,182794.64,20107.41,202902.05,
3,3174,Flucytosine 500 mg Tablet 100 Tablets,175701,false,false,false,,PPM-GF,GFATM,GF_2024,Reçu,220,2024-03-22,16500.00,1815.00,18315.00,
4,4583,Nevirapine 10 mg/mL Suspension w/ Syringe 100 mL,175715,false,false,false,,Govt,Govt,BGE_2023,Reçu,5010,2024-01-12,8767.50,964.42,9731.92,


In [11]:
# Fichier de mappage des informations des code produits SAGE X3 et QAT
df_map_prod_pnls = pd.read_csv("/home/jovyan/workspace/automating_dap_tools/data/stock_tracking_file/correspondance_produits_par_programme/map_produits_pnls.csv",
                               sep=';', encoding='utf-8')

df_map_prod_pnls.columns = df_map_prod_pnls.columns.str.replace('Ã©', 'é').str.replace('â', '').str.rstrip().str.lstrip()
df_map_prod_pnls.head()

,Standard product code,ID de produit QAT / Identifiant de produit (prévision),Produit (planification) / Produit (prévision)
0,3120042,3067,Female Condom (Nitrile) Lubricated 17 cm 100...
1,3120010,4806,Personal Lubricant (Water-Based) 4.5 g Topical...
2,3120010,4806,Personal Lubricant (Water-Based) 4.5 g Topical...
3,3120010,4806,Personal Lubricant (Water-Based) 4.5 g Topical...
4,3120044,4182,Male Condom (Latex) Lubricated No Logo 53 mm...


In [12]:
df_map_prod_pnls = df_map_prod_pnls.drop(columns='Produit (planification) / Produit (prévision)').drop_duplicates()

df_map_prod_pnls = df_map_prod_pnls.loc[(df_map_prod_pnls['Standard product code'].notna()) & (df_map_prod_pnls['Standard product code']!='ND')]

assert df_plan_approv.merge(
    df_map_prod_pnls,
    on='ID de produit QAT / Identifiant de produit (prévision)',
    how='left').shape[0] == df_plan_approv.shape[0]

df_plan_approv = df_plan_approv.merge(
    df_map_prod_pnls,
    on='ID de produit QAT / Identifiant de produit (prévision)',
    how='left')

In [13]:
%%time

wb_template = gstf.update_sheet_plan_approv(wb_template, df_plan_approv.copy())

CPU times: user 135 ms, sys: 3 µs, total: 135 ms
Wall time: 134 ms


# Mise à jour des données en se basant sur la source `Annexe 1 - Consolidation`

In [14]:
db_ops.reload_connection()

schema_name = "dap_tools"

* `df_produit`: dataframe récapitulant les produits pour le programme spécifique

In [15]:
# Il me faut d'abord réconstituer la base des données produit, par rapport au mois et au programme sélectionner

df_produit = pd.read_sql(
    f"""
    SELECT prod.*, st.stock_theorique_mois_precedent
    FROM {schema_name}.stock_track st
    INNER JOIN {schema_name}.dim_produit_stock_track prod ON st.id_produit_stock_track_fk = prod.id_produit_stock_track_pk
    WHERE prod.programme='{programme}' AND st.date_report = '{date_report_format}'""",
    db_ops.civ_engine).drop_duplicates()

df_produit.head(3)

,id_produit_stock_track_pk,nouveau_code,code,sous_groupes,designation_produit,categorie,cdtmt,unit_niveau_central,unit_niveau_peripherique,designation_acronym,code_qat,programme,stock_theorique_mois_precedent
0,45,3100001,AR33198,Médicaments,ABACAVIR / LAMIVUDINE 120/60 mg comp disp. bte/30,Produit traceur,30,BOITE/30,COMPRIME,ABC/3TC 120/60MG CP BTE/30,None,PNLS,70467.0
1,46,3100003,AR33196,Médicaments,ABACAVIR / LAMIVUDINE 600/300 mg comp. bte/30,Produit non traceur,30,BOITE/30,COMPRIME,None,None,PNLS,27734.0
2,47,3100005,AR33194,Médicaments,ABACAVIR / LAMIVUDINE/DOLUTEGRAVIR 600 mg / 30...,Produit non traceur,30,BOITE/30,None,None,None,PNLS,-656.0


* `df_dmm_global`: contient l'historique des dmm pour le programme spécifique

In [16]:
df_dmm_global = pd.read_sql(
    f"""
    SELECT prod.*, st_dmm.* FROM {schema_name}.stock_track_dmm st_dmm
    INNER JOIN {schema_name}.dim_produit_stock_track prod ON st_dmm.id_produit_stock_track_fk = prod.id_produit_stock_track_pk
    WHERE prod.programme='{programme}'""", db_ops.civ_engine)

df_dmm_global['date_report'] = pd.to_datetime(df_dmm_global['date_report'])

df_dmm_global.head(4)

,id_produit_stock_track_pk,nouveau_code,code,sous_groupes,designation_produit,categorie,cdtmt,unit_niveau_central,unit_niveau_peripherique,designation_acronym,code_qat,programme,id_stock_track_dmm_pk,id_produit_stock_track_fk,date_report,dmm,nbre_mois_consideres,distributions_mois_consideres,dmm_calculee_valider,dmm_validee_precedent,commentaire
0,45,3100001,AR33198,Médicaments,ABACAVIR / LAMIVUDINE 120/60 mg comp disp. bte/30,Produit traceur,30,BOITE/30,COMPRIME,ABC/3TC 120/60MG CP BTE/30,None,PNLS,749,45,2023-08-01,4293.0,NaN,NaN,NaN,NaN,None
1,46,3100003,AR33196,Médicaments,ABACAVIR / LAMIVUDINE 600/300 mg comp. bte/30,Produit non traceur,30,BOITE/30,COMPRIME,None,None,PNLS,750,46,2023-08-01,55.0,NaN,NaN,NaN,NaN,None
2,47,3100005,AR33194,Médicaments,ABACAVIR / LAMIVUDINE/DOLUTEGRAVIR 600 mg / 30...,Produit non traceur,30,BOITE/30,None,None,None,PNLS,751,47,2023-08-01,NaN,NaN,NaN,NaN,NaN,None
3,48,3100004,AR33195,Médicaments,ABACAVIR 300 mg comp. bte/60,Produit non traceur,60,BOITE/60,COMPRIME,None,None,PNLS,752,48,2023-08-01,0.0,NaN,NaN,NaN,NaN,None


* `df_cmm_global`: contient l'historique des cmm pour le programme spécifique

In [17]:
df_cmm_global = pd.read_sql(
    f"""
    SELECT prod.*, st_cmm.* FROM {schema_name}.stock_track_cmm st_cmm
    INNER JOIN {schema_name}.dim_produit_stock_track prod ON st_cmm.id_produit_stock_track_fk = prod.id_produit_stock_track_pk
    WHERE prod.programme='{programme}'""", db_ops.civ_engine)

df_cmm_global['date_report'] = pd.to_datetime(df_cmm_global['date_report'])

df_cmm_global.head(4)

,id_produit_stock_track_pk,nouveau_code,code,sous_groupes,designation_produit,categorie,cdtmt,unit_niveau_central,unit_niveau_peripherique,designation_acronym,code_qat,programme,id_stock_track_cmm_pk,id_produit_stock_track_fk,date_report,cmm,nbre_mois_consideres,distributions_mois_consideres,cmm_calculee_valider,cmm_validee_precedent,commentaire
0,45,3100001,AR33198,Médicaments,ABACAVIR / LAMIVUDINE 120/60 mg comp disp. bte/30,Produit traceur,30,BOITE/30,COMPRIME,ABC/3TC 120/60MG CP BTE/30,None,PNLS,749,45,2023-08-01,9752.0,NaN,NaN,NaN,NaN,None
1,46,3100003,AR33196,Médicaments,ABACAVIR / LAMIVUDINE 600/300 mg comp. bte/30,Produit non traceur,30,BOITE/30,COMPRIME,None,None,PNLS,750,46,2023-08-01,2970.0,NaN,NaN,NaN,NaN,None
2,47,3100005,AR33194,Médicaments,ABACAVIR / LAMIVUDINE/DOLUTEGRAVIR 600 mg / 30...,Produit non traceur,30,BOITE/30,None,None,None,PNLS,751,47,2023-08-01,NaN,NaN,NaN,NaN,NaN,None
3,48,3100004,AR33195,Médicaments,ABACAVIR 300 mg comp. bte/60,Produit non traceur,60,BOITE/60,COMPRIME,None,None,PNLS,752,48,2023-08-01,77.0,NaN,NaN,NaN,NaN,None


In [18]:
%%time
wb_template = gstf.update_sheet_annexe_1(wb_template, date_report, df_cmm_global, df_dmm_global, df_produit)

CPU times: user 7.33 s, sys: 20.1 ms, total: 7.35 s
Wall time: 7.35 s


# Mise à jour des données en se basant sur la source `Annexe 2 - Suivi de Stock`

In [19]:
%%time

wb_template = gstf.update_sheet_annexe_2(wb_template, df_plan_approv.copy(), date_report)

CPU times: user 1.45 s, sys: 9.02 ms, total: 1.46 s
Wall time: 1.46 s


# Mise à jour des données de la feuille `Repertoire Produit`

In [20]:
%%script false --no-raise-error

db_ops.civ_cursor.execute(f"""
    DROP TABLE IF EXISTS {schema_name}.stock_track_repertoire_prod CASCADE;
    CREATE TABLE {schema_name}.stock_track_repertoire_prod(
    id_stock_track_repertoire_prod_pk SERIAL PRIMARY KEY,
    programme_contract VARCHAR(20),
    sous_programme VARCHAR(250),
    code_pipeline VARCHAR(250),
    code_qat VARCHAR(250),
    standard_product_code VARCHAR(250),
    code_x_3 BIGINT,
    designation_produit TEXT,
    categorie VARCHAR(100),
    unit_niveau_central VARCHAR(250),
    unit_niveau_peripherique VARCHAR(250),
    conditionnement INTEGER,
    acronym VARCHAR(250),
    cout_unitaire REAL,
    programme VARCHAR(20) REFERENCES {schema_name}.dim_programme("Programme") ON DELETE CASCADE,
    CONSTRAINT unique_stock_track_repertoire_prod UNIQUE (id_stock_track_repertoire_prod_pk)
    );
""")

db_ops.conn.commit()

In [21]:
%%script false --no-raise-error

df_repertoire_prod = pd.read_excel("/home/jovyan/workspace/automating_dap_tools/data/stock_tracking_file/folder_by_month/Juin 2024/Fichier Suivi de Stock PNLS - Juin 2024.xlsx",
                                   sheet_name='Repertoire_Prod')

df_repertoire_prod.rename(columns={
    'Programme': 'programme_contract',
    'Sous-programme': 'sous_programme',
    'Code Pipeline': 'code_pipeline',
    'Standard product code': 'standard_product_code',
    'Code X3': 'code_x_3',
    'Désignation': 'designation_produit',
    'Categorie': 'categorie',
    'Unité niv central': 'unit_niveau_central',
    'Unité niv périphérique': 'unit_niveau_peripherique',
    'Conditionnement': 'conditionnement',
    'Acronym': 'acronym',
    'Cout Unitaire': 'cout_unitaire'
}, inplace=True
)

df_repertoire_prod.head(3)

In [22]:
%%script false --no-raise-error
df_repertoire_prod['code_x_3'] = df_repertoire_prod['code_x_3'].apply(lambda x: int(x) if not pd.isna(x) else x).astype(float)
df_repertoire_prod = df_repertoire_prod.map(lambda x: x.lstrip().rstrip() if isinstance(x, str) else x)
df_repertoire_prod['cout_unitaire'] = df_repertoire_prod['cout_unitaire'].replace('', np.NaN).astype(float)

In [23]:
%%script false --no-raise-error

df_repertoire_prod['programme'] = programme

df_repertoire_prod.drop_duplicates()\
                  .to_sql('stock_track_repertoire_prod', con = db_ops.civ_engine, schema=schema_name, index=False, if_exists='append')

db_ops.civ_engine.dispose()

In [24]:
%%script false --no-raise-error
df_repertoire_prod = pd.read_sql(f"SELECT * FROM {schema_name}.stock_track_repertoire_prod where programme='{programme}'", db_ops.civ_engine)

df_repertoire_prod = df_repertoire_prod[[col for col in df_repertoire_prod.columns if col not in ('programme', 'id_stock_track_repertoire_prod_pk', 'code_qat')]]

df_repertoire_prod.rename(
    columns={
        'programme_contract': 'Programme',
        'sous_programme': 'Sous-programme',
        'code_pipeline': 'Code Pipeline',
        'standard_product_code': 'Standard product code',
        'code_x_3': 'Code X3',
        'designation_produit': 'Désignation',
        'categorie': 'Categorie',
        'unit_niveau_central': 'Unité niv central',
        'unit_niveau_peripherique': 'Unité niv périphérique',
        'conditionnement': 'Conditionnement',
        'acronym': 'Acronym',
        'cout_unitaire': 'Cout Unitaire'
    }, inplace = True
)

df_repertoire_prod['Code X3'] = df_repertoire_prod['Code X3'].apply(lambda x: int(x) if not pd.isna(x) else x)
df_repertoire_prod.head()

In [25]:
%%script false --no-raise-error

ws_repertoire_prod = wb_template['Repertoire_Prod']

header_row = list(ws_repertoire_prod.iter_rows(max_row=1,max_col=gstf.column_index_from_string('L')))[0]
cols_df_rep_prod = list(df_repertoire_prod.columns)
dico_cols = {}

for cell in header_row:
    if cell.value.rstrip() in cols_df_rep_prod:
        dico_cols[cell.value.rstrip()] = [
            cell.column_letter, cell.col_idx,
            cols_df_rep_prod.index(cell.value.rstrip()),
            max([len(val) for val in df_repertoire_prod[cell.value.rstrip()].astype(str)])
        ]

font = gstf.Font(name='Arial Narrow', size=11)
alignment = gstf.Alignment(horizontal='left', vertical='center', wrap_text=True)
fill_first = gstf.PatternFill(start_color="FFE2EFDA", end_color="FFE2EFDA", fill_type="solid")
fill_second = gstf.PatternFill(start_color="FFB4C6E7", end_color="FFB4C6E7", fill_type="solid")
side = gstf.Side(style='thin', color='FF6eb4da')
border = gstf.Border(left=side, right=side, bottom=side)

number_format_cols = {
    'Cout Unitaire': '0.0',
    'Code X3': '0',
    'Conditionnement': '0',
}

for start, row in enumerate(gstf.dataframe_to_rows(df_repertoire_prod, index=False, header=False), start=1):
    for col, element in dico_cols.items():
        cell = ws_repertoire_prod.cell(row=start+1, column=element[1], value=row[element[2]])
        cell.font = font
        cell.border = border
        cell.alignment = alignment
        cell.fill = fill_first if start%2!=0 else fill_second
        if col in number_format_cols:
            cell.number_format = number_format_cols[col]

In [26]:
df_repertoire_prod = pd.read_sql(f"SELECT * FROM {schema_name}.stock_track_repertoire_prod where programme='{programme}'", db_ops.civ_engine)

In [27]:
%%time
wb_template = gstf.update_sheet_repertoire_prod(wb_template, df_repertoire_prod)

CPU times: user 269 ms, sys: 3.01 ms, total: 272 ms
Wall time: 270 ms


# Mise à jour des données en se basant sur la source `Prévision`

In [28]:
%time

wb_template = gstf.update_sheet_prevision(wb_template, date_report, df_produit.loc[df_produit.categorie == 'Produit traceur', ['nouveau_code', 'designation_produit']])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


## Mise à jour de certains éléments de la feuille `Rapport`

In [29]:
ws_rapport = wb_template['Rapport']

In [30]:
ws_rapport['D4'].value = programme
ws_rapport['D6'].value = gstf.get_current_variable(date_report)[1].split()[0].capitalize()
ws_rapport['E6'].value = gstf.get_current_variable(date_report)[1].split()[1]

# Exportation du `Workbook`

In [31]:
%%time
wb_template.save(
    filename=f"/home/jovyan/workspace/automating_dap_tools/data/stock_tracking_file/Export Fichier Suivi de Stock/Fichier Suivi de Stock {programme}.xlsx"
)

CPU times: user 8.15 s, sys: 211 ms, total: 8.36 s
Wall time: 9.75 s


# Draft

In [30]:
# wb_template = pyxl.load_workbook(
#     filename="/home/jovyan/workspace/automating_dap_tools/data/stock_tracking_file/Export Fichier Suivi de Stock/Fichier Suivi de Stock PNLS.xlsx"
# )

In [31]:
%%script false --no-raise-error

df_produit_stock = pd.read_sql(f"SELECT * FROM {schema_name}.dim_produit_stock_track where programme='{programme}'", db_ops.civ_engine)

df_produit_stock.head(2)

In [32]:
# Ce qui était fait avant
# df_repertoire_prod = df_repertoire_prod.loc[df_repertoire_prod.code_x_3.notna()].drop_duplicates()

# df_repertoire_prod['code_qat'] = df_repertoire_prod['code_qat'].astype(float)

# assert df_plan_approv.merge(
#     df_repertoire_prod[['code_qat', 'code_x_3']],
#     left_on='ID de produit QAT / Identifiant de produit (prévision)', right_on='code_qat',
#     how='left').shape[0] == df_plan_approv.shape[0]

# df_plan_approv = df_plan_approv.merge(
#     df_repertoire_prod[['code_qat', 'code_x_3']],
#     left_on='ID de produit QAT / Identifiant de produit (prévision)', right_on='code_qat',
#     how='left')

# df_plan_approv.drop(columns=['code_qat'], inplace=True)
# df_plan_approv.loc[df_plan_approv.code_x_3.notna()].head(3)

# df_produit_stock['code_qat'] = df_produit_stock['code_qat'].astype(float)

# assert df_plan_approv.merge(
#     df_produit_stock[['code_qat', 'nouveau_code']].rename(columns={'nouveau_code':'code_x_3'}),
#     left_on='ID de produit QAT / Identifiant de produit (prévision)', right_on='code_qat',
#     how='left').shape[0] == df_plan_approv.shape[0]

# df_plan_approv = df_plan_approv.merge(
#     df_produit_stock[['code_qat', 'nouveau_code']].rename(columns={'nouveau_code':'code_x_3'}),
#     left_on='ID de produit QAT / Identifiant de produit (prévision)', right_on='code_qat',
#     how='left')

# df_plan_approv.drop(columns=['code_qat'], inplace=True)

# df_plan_approv.loc[df_plan_approv.code_x_3.notna()].head(3)

In [33]:
assert df_plan_approv.merge(
    df_map_prod_pnls,
    on='ID de produit QAT / Identifiant de produit (prévision)',
    how='left').shape[0] == df_plan_approv.shape[0]

df_plan_approv = df_plan_approv.merge(
    df_map_prod_pnls,
    on='ID de produit QAT / Identifiant de produit (prévision)',
    how='left')

In [34]:
%%script false --no-raise-error

df_plan_approv.rename(
    columns={
        'code_x_3': 'Standard product code',
        'Agent d`approvisionnement': "Centrale d'achat",
        'Source de financement': "Source Financement",
        "Produit (planification) / Produit (prévision)": 'Produits',
        'date de réception': 'DATE',
        'Quantité': 'Quantite',
        'État': 'Status',
        'Coût unitaire de produit (USD)': 'Cout des Produits',
        'Coût du fret (USD)': 'Couts du fret',
        'Coût total (USD)': 'Couts totaux',
    }, inplace=True)

# Mise à jour de la feuille plan d'approvisionement
ws_plan_approv = wb_template['Plan dappro']

header_row = list(ws_plan_approv.iter_rows(max_col=gstf.column_index_from_string('Q')))[0]

dico_formules = {
    'E': "=IFERROR(VLOOKUP(A{start},Repertoire_Prod!$E${start}:$G$315,3,FALSE),\"\")", #'categorie'
    'L': "=IF(AND(H{start}=\"planifié\",(F{start}-TODAY())<90),\"Risk of delay\",\"Ok\")", #'Alert?'
    'M': "=IFERROR(VLOOKUP(A{start},Repertoire_Prod!$E${start}:$K$315,7,FALSE),\"\")", #'Acronym'
    'N': "=TEXT(F{start},\"mmm\") & \"-\" &  YEAR(F{start})", # 'Date updated'
    'O': "=IF(H{start}=\"\",\"\",IF(H{start}=\"Reçu\", 1, 0))", #'Received?'
    'P': "=A{start}&\"_\"&F{start}", # concatenation du code et de la date
}

cols_df_plan_approv = list(df_plan_approv.columns)
dico_cols = {}

for cell in header_row:
    if cell.value is None:
        continue
    if cell.value.rstrip().lstrip() in cols_df_plan_approv:
        dico_cols[cell.value.rstrip().lstrip()] = [
            cell.column_letter, cell.col_idx,
            cols_df_plan_approv.index(cell.value.rstrip().lstrip()),
            max([len(val) for val in df_plan_approv[cell.value.rstrip().lstrip()].astype(str)])
        ]

In [35]:
%%script false --no-raise-error

font = gstf.Font(name='Arial Narrow', size=11)
fill = gstf.PatternFill(start_color="FFD9E2F3", fill_type="solid")
alignment = gstf.Alignment(horizontal='left', vertical='center')
alignment_two = gstf.alignment_two
date_style = gstf.NamedStyle(name='date_style_pa', number_format='DD/MM/YYYY')
number_format_cols = {
    'Standard product code': '0',
    'Quantite': '0',
    'Cout des Produits': '0.00',
    'Couts du fret': '0.00',
    'Couts totaux': '0.00'
}
for start, row in enumerate(gstf.dataframe_to_rows(df_plan_approv, index=False, header=False), start=2):
    for col, element in dico_cols.items():
        cell = ws_plan_approv.cell(row=start, column=element[1], value= row[element[2]])
        cell.font = font
        cell.border = gstf.border
        cell.alignment = gstf.alignment_two if col not in ('Produits', 'Status') else alignment
        if col in ('Standard product code', 'DATE', 'Status'):
            cell.fill = fill 
        if col=='DATE':
            cell.style = date_style
            cell.border = gstf.border
        if col in number_format_cols:
            cell.number_format = number_format_cols[col]

    for col, formula in dico_formules.items():
        cell = ws_plan_approv.cell(row=start, column=gstf.column_index_from_string(col), value=formula.format(start=start))
        cell.font = font
        cell.alignment = alignment
        cell.border = gstf.border